# World University Rankings Data

________________________________________________________________________________________________________________

In [2]:
from selenium import webdriver  # Импортируем WebDriver для управления браузером
from selenium.webdriver.common.by import By  # Импортируем локаторы (By.XPATH, By.CSS_SELECTOR и т.д.)
from selenium.webdriver.support.ui import WebDriverWait  # Явные ожидания
from selenium.webdriver.support import expected_conditions as EC  # Готовые условия для ожиданий
import time  # Паузы между действиями (иногда полезны для динамических страниц)

In [3]:
url = 'https://www.timeshighereducation.com/world-university-rankings/latest/world-ranking' # URL страницы с рейтингами университетов

То что внизу написано не ошибка, я просто добавила комментарии, а поом остановила выполнение кода, если его запустить и дождаться конца выполнения, все будет работать:)

Этот код открывает страницу в Chrome и, если появляется баннер с cookies, пытается нажать кнопку **“Allow all”**, чтобы он не мешал. Затем он ждёт, пока на странице реально появится таблица (хотя бы одна строка), потому что данные могут подгружаться не сразу.

Дальше начинается сбор данных из таблицы. Код много раз подряд (до 2000 итераций) берёт все видимые строки таблицы, вытаскивает текст из каждой ячейки и дополнительно пытается достать ссылку на страницу вуза из второй колонки. Чтобы не собирать одно и то же по несколько раз, он хранит все уже встреченные ссылки в `seen_hrefs` и добавляет строку в общий список `all_data` только если ссылка новая.

После каждого прохода он проверяет, появились ли вообще новые уникальные строки. Если на протяжении 25 итераций подряд ничего нового не добавляется, значит таблица больше не подгружается — и цикл останавливается. Чтобы подгрузка происходила, код ещё и постепенно скроллит таблицу вниз (скроллит к очередной строке), потому что на многих сайтах новые строки появляются только при прокрутке. В конце он закрывает браузер и выводит, сколько уникальных университетов получилось собрать.

In [4]:
driver = webdriver.Chrome()  # Запускаем ChromeDriver (браузер Chrome под управлением Selenium)
driver.maximize_window()  # Разворачиваем окно на весь экран (это не обязательно, но иногда помогает с отображением элементов)
driver.get(url)  # Открываем страницу по заранее заданному URL

# Пытаемся закрыть/принять окно cookie, если оно появляется
try:  # Баннер может не появиться — это нормальный кейс
    accept_btn = WebDriverWait(driver, 10).until(  # Ждём до 10 секунд
        EC.element_to_be_clickable(  # Пока кнопка не станет кликабельной
            (By.XPATH, "//button[contains(text(), 'Allow all')]")  # Кнопка с текстом "Allow all"
        )
    )
    accept_btn.click()  # Обычный клик по кнопке
    time.sleep(2)  # Даём странице закрыть баннер и обновиться
except Exception:  # Если баннера нет/не успел появиться/кнопка не нашлась
    pass  # Просто продолжаем выполнение

# Ждём, пока в таблице появятся строки (динамический контент может подгружаться не сразу)
WebDriverWait(driver, 10).until(  # Ждём до 10 секунд
    EC.presence_of_element_located(  # Условие: элемент присутствует в DOM
        (By.CSS_SELECTOR, "table tbody tr")  # Ищем хотя бы одну строку таблицы
    )
)

all_data = []  # Сюда будем складывать все собранные строки таблицы
seen_hrefs = set()  # Множество для уникальности (чтобы не добавлять одинаковые ссылки повторно)

last_new_count = 0  # Запоминаем количество уникальных строк после предыдущей итерации
no_change_count = 0  # Считаем, сколько итераций подряд не появляется новых строк
scroll_index = 0  # Индекс строки, к которой будем скроллить (для постепенного просмотра таблицы)

# Основной цикл: многократно читаем таблицу, пока подгружаются новые строки
for attempt in range(2000):  # Максимум 2000 проходов (защита от бесконечного цикла)
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")  # Берём все строки таблицы, которые сейчас видны

    # Проходим по каждой строке и пытаемся извлечь данные
    for row in rows:  # Идём по каждой найденной строке
        cells = row.find_elements(By.TAG_NAME, "td")  # Находим все ячейки <td> внутри строки
        if len(cells) == 0:  # Если ячеек нет (на всякий случай)
            continue  # Пропускаем строку

        row_data = [cell.text.strip() for cell in cells]  # Собираем текст каждой ячейки и чистим пробелы по краям

        if not any(row_data):  # Если все ячейки пустые (строка без данных)
            continue  # Пропускаем такую строку

        # Пытаемся достать ссылку на страницу вуза из второй колонки (cells[1])
        try:  # Ссылка может отсутствовать, поэтому оборачиваем в try
            link_element = cells[1].find_element(By.TAG_NAME, "a")  # Ищем <a> внутри второй ячейки
            uni_url = link_element.get_attribute("href")  # Берём атрибут href (URL страницы)
        except:  # Если ссылки нет или структура другая
            uni_url = ""  # Ставим пустую строку

        # Добавляем строку только если у неё есть ссылка и мы ещё не добавляли такую ссылку
        if uni_url and uni_url not in seen_hrefs:  # Проверяем: ссылка непустая и новая
            seen_hrefs.add(uni_url)  # Запоминаем ссылку как уже встреченную
            row_data.append(uni_url)  # Добавляем ссылку как последнюю колонку к данным строки
            all_data.append(row_data)  # Сохраняем строку в общий список

    current_total = len(all_data)  # Сколько уникальных строк уже собрано всего

    # Проверяем, появились ли новые строки по сравнению с прошлой итерацией
    if current_total > last_new_count:  # Если стало больше — значит добавили что-то новое
        last_new_count = current_total  # Обновляем счётчик "последнее количество"
        no_change_count = 0  # Сбрасываем счётчик "без изменений"
    else:  # Если новых строк не добавилось
        no_change_count += 1  # Увеличиваем число итераций без изменений
        if no_change_count >= 25:  # Если 25 раз подряд ничего нового
            break  # Выходим из цикла: скорее всего, подгрузка закончилась

    # Плавно скроллим: это может триггерить подгрузку новых строк, если таблица виртуализирована
    if len(rows) > 10:  # Если строк достаточно, чтобы скролл имел смысл
        target_index = min(len(rows) - 5, scroll_index)  # Выбираем индекс строки, к которой скроллим (не в самый низ)
        if target_index < len(rows):  # Проверяем, что индекс в пределах массива
            driver.execute_script(  # Выполняем JS
                "arguments[0].scrollIntoView({block: 'center', inline: 'nearest'});",  # Скроллим элемент в центр окна
                rows[target_index]  # Передаём строку, к которой нужно проскроллить
            )
        scroll_index += 3  # Сдвигаем скролл дальше, чтобы постепенно проходить по таблице

    time.sleep(1)  # Пауза между итерациями (даём странице время подгрузить данные)

driver.quit()  # Закрываем браузер и завершаем сессию WebDriver
print(f"Всего университетов: {len(all_data)}")  # Итоговый вывод количества уникальных строк


KeyboardInterrupt: 

In [ ]:
# Сохранить в CSV
import pandas as pd
df = pd.DataFrame(all_data)
df.columns = ["Rank", "Name", "Overall", "Teaching", "Research Environment", "Research Quality", "Industry", "International Outlook", "URL"]
df.to_csv("the_rankings_2026_full.csv", index=False, encoding='utf-8')